In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
import timm

In [2]:
from dataprocessing import DataLoaderWrapper

data=DataLoaderWrapper(batch_size=32, image_size=224)
celebA_train,celebA_val,celebA_test=data.initialize_celebA_dataloaders()
adience=data.initialize_adience_dataloaders()

\

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/adience



/

...

\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/celeb-a-test



/

hub://activeloop/celeb-a-test loaded successfully.



/Users/tulipmajumder/opt/anaconda3/lib/python3.9/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [ ]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)

model.eval()  # Set the model to evaluation mode

# Prepare to collect the predictions
all_probabilities = []

with torch.no_grad():
    for test_images, age_labels, gender_labels in celebA_val:
        out = model(test_images)
        
        # Apply softmax to calculate probabilities
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        
        # Collect the probabilities for further processing or analysis
        all_probabilities.append(probabilities)
        
#probabilities = torch.nn.functional.softmax(out[0], dim=0)

In [5]:
num_classes = 2  # 2 classes: male and female
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)       

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [7]:
num_epochs = 5

# Training loop
model.train()
for epoch in range(num_epochs):
    for images, genders in adience:  # Assuming the dataloader returns images and age labels
        images = images.to(device)
        genders = genders.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, genders)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_losses = []
    val_accuracies = []
    
    with torch.no_grad():  # Disable gradient calculation
        for val_images, val_age_labels in adience_val:
            val_images = val_images.to(device)
            val_age_labels = val_age_labels.to(device)

            # Forward pass
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_age_labels)
            val_losses.append(val_loss.item())

            # Calculate accuracy
            _, val_preds = torch.max(val_outputs, 1)
            val_accuracy = accuracy_score(val_age_labels.cpu(), val_preds.cpu())
            val_accuracies.append(val_accuracy)

# Save the trained model
torch.save(model.state_dict(), 'adience_vit_model.pth')

ValueError: too many values to unpack (expected 2)